In [7]:
import pyarrow.parquet as pq
import pandas as pd
from IPython.display import display

#dataset_1 = pq.ParquetDataset('dataset-1/w32processes')
#dataset_2 = pq.ParquetDataset('dataset-2/w32processes')
#dataset_3 = pq.ParquetDataset('dataset-3/w32processes')
dataset_4 = pq.ParquetDataset('../dataset-4/w32processes')

#table_1 = dataset_1.read()
#table_2 = dataset_2.read()
#table_3 = dataset_3.read()
table_4 = dataset_4.read()

#df_1 = table_1.to_pandas()
#df_2 = table_2.to_pandas()
#df_3 = table_3.to_pandas()
df = table_4.to_pandas()

#df_1.to_csv('test_1.csv')
#df_2.to_csv('test_2.csv')
#df_3.to_csv('test_3.csv')
#df_4.to_csv('test_4.csv')

pd.set_option('display.max_colwidth', None)

print('Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3')

Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3


In [8]:
#display(df)

In [9]:
n_elevated_processes = df[df['username'].str.contains('NT AUTHORITY')]
print('Nº of elevated processes: ' + str(len(n_elevated_processes)))

Nº of elevated processes: 4346


In [10]:
# Elevated powershell / cmd processes:
elevated_consoles = n_elevated_processes[n_elevated_processes['name'].str.contains('cmd')]
print('Nº of elevated cmds: ' + str(len(elevated_consoles)))

display(elevated_consoles)

Nº of elevated cmds: 8


,hostname,name,arguments,username,path,pid
61,EC0002LKS,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Program Files (x86)\Common Files\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,5968
54,EE0006XXF,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Program Files (x86)\Common Files\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,252
7,NC0009YBY,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,2492
16,SE0006JXV,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,3172
102,SE0006JXV,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Users\masonwood4\Documents\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,3424
74,SE0007JXV,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Windows\servicing\LCUrun_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,596
93,SN0007ZVK,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,5368
36,WW0006YQW,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,1476


In [11]:
elevated_consoles = n_elevated_processes[n_elevated_processes['name'].str.contains('powershell')]
print('Nº of elevated cmds: ' + str(len(elevated_consoles)))

display(elevated_consoles)

Nº of elevated cmds: 0


,hostname,name,arguments,username,path,pid


In [16]:
elevated = df[df['username'].str.upper().str.contains('SYSTEM|NT AUTHORITY')]


host_count = elevated.groupby('name')['hostname'].nunique()

# Less than 10 hosts
rare_procs = host_count[host_count < 10].index.tolist()
rare_elevated = elevated[elevated['name'].isin(rare_procs)]

display(rare_elevated)

,hostname,name,arguments,username,path,pid
61,EC0002LKS,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Program Files (x86)\Common Files\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,5968
54,EE0006XXF,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Program Files (x86)\Common Files\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,252
7,NC0009YBY,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,2492
16,SE0006JXV,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,3172
102,SE0006JXV,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Users\masonwood4\Documents\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,3424
74,SE0007JXV,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Windows\servicing\LCUrun_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,596
93,SN0007ZVK,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,5368
36,WW0006YQW,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,1476


In [17]:
# Posibles servicios de Windows
service_like = df[
    df['name'].str.lower().isin(['services.exe', 'svchost.exe']) |
    df['path'].str.lower().str.contains(r'\\system32\\services.exe|\\system32\\svchost.exe', na=False)
]

service_like_elevated = service_like[service_like['username'].str.upper().str.contains('SYSTEM|NT AUTHORITY')]
service_host_counts = service_like_elevated.groupby('name')['hostname'].nunique()

# Less than 10 hosts
rare_services = service_host_counts[service_host_counts < 10].index.tolist()
susp_services = service_like_elevated[service_like_elevated['name'].isin(rare_services)]

display(susp_services)

,hostname,name,arguments,username,path,pid
